## Exploratory Data-Analysis



### Dependencies

In [53]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore', message='The default value of regex will change')
import re
import ipywidgets as widgets

In [54]:
df = pd.read_csv('/content/drive/MyDrive/♻️ Obsolete ♻️/RS_A1_Datasets/cuisine_updated.csv')

In [55]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Raw Dataframe 

In [56]:
df.head()

,name,image_url,description,cuisine,course,diet,prep_time,ingredients,instructions,image_available
0,Doddapatre Tambuli Recipe (Karpooravalli Thayi...,https://www.archanaskitchen.com/images/archana...,Doddapatre Tambuli Recipe is a popular healing...,Indian,Side Dish,Vegetarian,Total in 30 M,\n\n\t\t\t\t\t\t\t \t\t4 cups Indian borage (D...,"To begin making the Doddapatre Tambuli, first ...",1
1,Fish Tandoori Recipe,https://www.archanaskitchen.com/images/archana...,Have you ever tried out fish tandoori? Trust m...,North Indian Recipes,Appetizer,High Protein Non Vegetarian,Total in 80 M,\n\n\t\t\t\t\t\t\t \t\t1 Fish\t\t\t\t\t\t\t \t...,"To begin making the Fish Tandoori recipe, keep...",1
2,Arbi Shimla Mirch Sabzi Recipe – Colocasia Cap...,https://www.archanaskitchen.com/images/archana...,Arbi Shimla Mirch Sabzi Recipe is a variation ...,North Indian Recipes,Main Course,No Onion No Garlic (Sattvic),Total in 55 M,\n\n\t\t\t\t\t\t\t \t\t300 grams Colocasia roo...,To begin making the Arbi Shimla Mirch Sabzi Re...,1
3,Ambur Style Brinjal Curry Recipe,https://www.archanaskitchen.com/images/archana...,Brinjal Curry is prepared in different ways in...,Indian,Lunch,Vegetarian,Total in 20 M,\n\n\t\t\t\t\t\t\t \t\t5 Brinjal (Baingan / Eg...,To begin making the Ambur Style Brinjal Curry ...,1
4,Mavinakayi Menasinakai Curry Recipe - Raw Man...,https://www.archanaskitchen.com/images/archana...,Mangalorean Mavinakayi Menasinakai Curry Recip...,Mangalorean,Side Dish,Vegetarian,Total in 35 M,\n\n\t\t\t\t\t\t\t \t\t1 cup Mango (Raw)\t\t\t...,To begin making Mangalorean Mavinakayi Menasin...,1


### Clean Dataframe

In [57]:
df = df[['name','cuisine','course','diet','prep_time','ingredients','instructions']]
df['prep_time'] = df['prep_time'].fillna('0').str.replace('Total in ', '').str.replace(' M', '')
df = df[df['prep_time'].str.isnumeric()]
df['prep_time'] = df['prep_time'].astype(int)
df['ingredients'] = df['ingredients'].replace(['\n','\t'],'',regex=True)
df.head()

<ipython-input-57-1adb62de7d72>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['prep_time'] = df['prep_time'].fillna('0').str.replace('Total in ', '').str.replace(' M', '')


,name,cuisine,course,diet,prep_time,ingredients,instructions
0,Doddapatre Tambuli Recipe (Karpooravalli Thayi...,Indian,Side Dish,Vegetarian,30,"4 cups Indian borage (Doddapatre) , cleaned a...","To begin making the Doddapatre Tambuli, first ..."
1,Fish Tandoori Recipe,North Indian Recipes,Appetizer,High Protein Non Vegetarian,80,"1 Fish , nicely washed with bones on (I used ...","To begin making the Fish Tandoori recipe, keep..."
2,Arbi Shimla Mirch Sabzi Recipe – Colocasia Cap...,North Indian Recipes,Main Course,No Onion No Garlic (Sattvic),55,300 grams Colocasia root (Arbi) 1 Green Bell...,To begin making the Arbi Shimla Mirch Sabzi Re...
3,Ambur Style Brinjal Curry Recipe,Indian,Lunch,Vegetarian,20,5 Brinjal (Baingan / Eggplant) 1 Onion 2 To...,To begin making the Ambur Style Brinjal Curry ...
4,Mavinakayi Menasinakai Curry Recipe - Raw Man...,Mangalorean,Side Dish,Vegetarian,35,"1 cup Mango (Raw) , diced 2 tablespoons Jagg...",To begin making Mangalorean Mavinakayi Menasin...


### List of Distinct Values to Group By

In [58]:
cuisine_distinct = df['cuisine'].unique().tolist()
for i, j in enumerate(cuisine_distinct, start=1):
    print(f"{i}. {j}")

1. Indian
2. North Indian Recipes
3. Mangalorean
4. Kerala Recipes
5. Konkan
6. South Indian Recipes
7. Tamil Nadu
8. Bengali Recipes
9. Andhra
10. Coorg
11. Hyderabadi
12. Kashmiri
13. Punjabi
14. Gujarati Recipes﻿
15. Coastal Karnataka
16. Karnataka
17. Goan Recipes
18. Sichuan
19. Udupi
20. Chettinad
21. Maharashtrian Recipes
22. Mughlai
23. Malvani
24. North East India Recipes
25. Rajasthani
26. Parsi Recipes
27. Bihari
28. Awadhi
29. North Karnataka
30. Assamese
31. Pakistani
32. Himachal
33. Middle Eastern
34. Oriya Recipes
35. Uttarakhand-North Kumaon 
36. Lucknowi
37. Sindhi
38. Indo Chinese
39. Uttar Pradesh
40. Kongunadu
41. Chinese
42. Haryana
43. South Karnataka
44. Fusion
45. nan
46. Malabar
47. Nepalese
48. Continental
49. Jharkhand
50. Sri Lankan
51. Asian
52. Thai
53. Arab
54. Nagaland
55. Afghan
56. African
57. Korean


In [59]:
course_distinct = df['course'].unique().tolist()
for i, j in enumerate(course_distinct, start=1):
    print(f"{i}. {j}")

1. Side Dish
2. Appetizer
3. Main Course
4. Lunch
5. Dinner
6. Dessert
7. Indian Breakfast
8. nan
9. North Indian Breakfast
10. South Indian Breakfast
11. World Breakfast
12. One Pot Dish
13. Snack
14. Brunch


In [60]:
diet_distinct = df['diet'].unique().tolist()
for i, j in enumerate(diet_distinct, start=1):
    print(f"{i}. {j}")

1. Vegetarian
2. High Protein Non Vegetarian
3. No Onion No Garlic (Sattvic)
4. Non Vegeterian
5. Eggetarian
6. Diabetic Friendly
7. High Protein Vegetarian
8. Gluten Free
9. nan
10. Vegan


### Sorted Dataframes

In [61]:
sorted_df = df.sort_values(by=['cuisine'])
print(sorted_df[['cuisine','name']].to_string(index=False))

                  cuisine                                                                                                        name
                   Afghan                                                   Lavand-E-Murgh Recipe - Afghani Chicken In Yoghurt Gravy 
                  African                                                                Ras El Hanout Recipe (Moroccan Spice Powder)
                   Andhra                                                         Andhra Style Tamati Pachadi Recipe (Tomato Chutney)
                   Andhra                                              Beerakaya Tomato Koora Recipe (Andhra Style Ridge Gourd Curry)
                   Andhra                                    Vankaya Gasagasala Kura | Andhra Style Eggplant Poppy Seeds Curry Recipe
                   Andhra                                      आंध्रा स्टाइल मूंगफली चटनी रेसिपी - Andhra Style Peanut Chutney Recipe
                   Andhra                                     

In [62]:
sorted_df = df.sort_values(by=['course'])
print(sorted_df[['course','name']].to_string(index=False))

                course                                                                                                        name
             Appetizer                                                                      Chicken Cooked in Banana Leaves Recipe
             Appetizer                                                        Rajasthani Gunde Ka Achaar Recipe (Bird Lime Pickle)
             Appetizer                                               Chettinad Chicken Recipe | Pepper Chicken Dry | South Indian 
             Appetizer                                                  Chettinad Chicken Roast Recipe | Chicken Ghee Roast Recipe
             Appetizer                                                          Chicken Dimsums Recipe - Steamed Chicken Dumplings
             Appetizer                                        Smoked Tandoori Chicken Recipe - Classic North Indian Chicken Recipe
             Appetizer                                                             

In [63]:
sorted_df = df.sort_values(by=['diet'])
print(sorted_df[['diet','name']].to_string(index=False))

                        diet                                                                                                        name
           Diabetic Friendly                                                           Ragi Kuzhi Paniyaram Recipe with Ragi Dosa Batter
           Diabetic Friendly                                                      Kathal Masaledar Sabzi Recipe - Jackfruit Masala Sabzi
           Diabetic Friendly                                                   Ragi & Oats Rava Idli Recipe-Finger Millet and Oats Idli 
           Diabetic Friendly                                       लौकी टमाटर की सब्ज़ी रेसिपी - Lauki Tamatar Ki Sabzi (Recipe In Hindi)
           Diabetic Friendly                                                                   Foxtail and Jowar Millet Paniyaram Recipe
           Diabetic Friendly                                                     लौकी चना दाल रेसिपी - Lauki Chana Dal (Recipe In Hindi)
           Diabetic Friendly             

In [64]:
sorted_df = df.sort_values(by=['prep_time'])
print(sorted_df[['prep_time','name']].to_string(index=False))

 prep_time                                                                                                        name
         0                                                   लहसुनि पुदीना चटनी रेसिपी - Lehsuni Pudina Chutney Recipe
         0                                                                          Bisi Bele Bath Spice Powder Recipe
         0                                                                                Malvani Mutton Curry Recipe 
         0                                                                Konkani Style Koli Masala Recipe- Spice Mix 
         0                                                                                     Besan Oats Ladoo Recipe
         0                                             सब्जी वाली रवा इडली रेसिपी - Veggie Rava Idli (Recipe In Hindi)
         0                                                          कढ़ी पकोड़ा रेसिपी - Kadhi Pakoras (Recipe In Hindi)
         0                               Urulaik

In [65]:
# Clean the 'ingredients' column and create a new column 'ingredients_clean'
df['ingredients_clean'] = df['ingredients'].str.replace('\d+g\s+', '').str.replace('\d+\s+\(.*?\)\s+', '').str.replace('\s+\(.+?\)\s+', '').str.replace('\s+\d+\s+', '').str.replace('\s+nicely\s+washed\s+with\s+bones\s+', '').str.strip()

### Widgets (Experimental)

In [66]:
widgets.Dropdown(
    options=['Indian', 'North Indian Recipes', 'Mangalorean', 'Kerala Recipes', 'Konkan', 'South Indian Recipes', 'Tamil Nadu', 'Bengali Recipes', 'Andhra', 'Coorg', 'Hyderabadi', 'Kashmiri', 'Punjabi', 'Gujarati Recipes\ufeff', 'Coastal Karnataka', 'Karnataka', 'Goan Recipes', 'Sichuan', 'Udupi', 'Chettinad', 'Maharashtrian Recipes', 'Mughlai', 'Malvani', 'North East India Recipes', 'Rajasthani', 'Parsi Recipes', 'Bihari', 'Awadhi', 'North Karnataka', 'Assamese', 'Pakistani', 'Himachal', 'Middle Eastern', 'Oriya Recipes', 'Uttarakhand-North Kumaon ', 'Lucknowi', 'Sindhi', 'Indo Chinese', 'Uttar Pradesh', 'Kongunadu', 'Chinese', 'Haryana', 'South Karnataka', 'Fusion', 'Malabar', 'Nepalese', 'Continental', 'Jharkhand', 'Sri Lankan', 'Asian', 'Thai', 'Arab', 'Nagaland', 'Afghan', 'African', 'Korean'],
    value='Indian',
    description='Cuisine:',
    disabled=False,
)

Dropdown(description='Cuisine:', options=('Indian', 'North Indian Recipes', 'Mangalorean', 'Kerala Recipes', '…

In [67]:
widgets.Dropdown(
    options=['Side Dish', 'Appetizer', 'Main Course', 'Lunch', 'Dinner', 'Dessert', 'Indian Breakfast', 'North Indian Breakfast', 'South Indian Breakfast', 'World Breakfast', 'One Pot Dish', 'Snack', 'Brunch'],
    value='Side Dish',
    description='Course:',
    disabled=False,
)

Dropdown(description='Course:', options=('Side Dish', 'Appetizer', 'Main Course', 'Lunch', 'Dinner', 'Dessert'…

In [68]:
widgets.Dropdown(
    options=
['Vegetarian', 'High Protein Non Vegetarian', 'No Onion No Garlic (Sattvic)', 'Non Vegeterian', 'Eggetarian', 'Diabetic Friendly', 'High Protein Vegetarian', 'Gluten Free', 'Vegan'],
    value='Vegetarian',
    description='Diet:',
    disabled=False,
)

Dropdown(description='Diet:', options=('Vegetarian', 'High Protein Non Vegetarian', 'No Onion No Garlic (Sattv…

In [69]:
widgets.FloatSlider(
    value=7.5,
    min=0,
    max=10.0,
    step=0.1,
    description='Prep-Time:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)


FloatSlider(value=7.5, continuous_update=False, description='Prep-Time:', max=10.0, readout_format='.1f')

## 0. Menu

In [81]:
    print("Welcome to the Recipe Recommender System!\n")
    print("Select the option from 1-10 based upon your preference:")
    print("1. Cuisine Based Recommender System")
    print("2. Course Based Recommender System")
    print("3. Diet Based Recommender System")
    print("4. Preparation Based Recommender System")
    print("5. Ingredients and Prep-Time Based Recommender System")
    print("6. Allergy Free and Cuisine Based Recommender System")                            
    print("7. Dietary-Restriction and Cuisine Based Recommender System")
    print("8. Utensil-Restriction and Cuisine Based Recommender System")
    print("9. Preparation Time and Cuisine Based Recommender System")
    print("10. Course and Cuisine Based Recommender System\n")
    option = input("Enter your choice (1-10): ")
    print("\n")

    if option == "1":
        cuisine = input("Enter the cuisine of your choice: \n")
        recommended_items = cuisine_based_recommender(cuisine)
        counter = 1
        for item, instructions in recommended_items.items():
            print(str(counter) + '. ' + item + ': ' + instructions + '\n')
            counter += 1
        

    elif option == "2":
        course = input("Enter the course of your choice: \n")
        recommended_items = course_based_recommender(course)
        counter = 1
        for item, instructions in recommended_items.items():
            print(str(counter) + '. ' + item + ': ' + instructions + '\n')
            counter += 1
        
    elif option == "3":
        diet = input("Enter your diet preference: \n")
        recommended_items = diet_based_recommender(diet)
        counter = 1
        for item, instructions in recommended_items.items():
            print(str(counter) + '. ' + item + ': ' + instructions + '\n')
            counter += 1
        

    elif option == "4":
        prep_time = int(input("Enter the time of preparation you prefer: \n"))
        recommended_items = prep_based_recommender(prep_time)
        counter = 1
        for item, instructions in recommended_items.items():
            print(str(counter) + '. ' + item + ': ' + instructions + '\n')
            counter += 1
            
        
    elif option == "5":
        #ingredients are like '100g pasta, 2 cups tomato sauce, 3 onion, 2 garlic cloves'
        ingredients = input("Enter the ingredients of your choice: \n") 
        prep_time = int(input("Enter the time of preparation you prefer: \n"))
        recommended_items = ingredients_based_recommender(ingredients,prep_time)
        counter = 1
        for item, instructions in recommended_items.items():
            print(str(counter) + '. ' + item + ': ' + instructions + '\n')
            counter += 1
        
        
    elif option == "6":
        cuisine = input("Enter the cuisine of your choice: \n")
        allergen= input("Enter the food you are allergic to: \n") #i.e: you may be allergic to "sugar" or "salt" etc
        recommended_items = allergy_based_recommender(cuisine,allergen)
        counter = 1
        for item, instructions in recommended_items.items():
            print(str(counter) + '. ' + item + ': ' + instructions + '\n')
            counter += 1

     
    elif option == "7":
        cuisine = input("Enter the cuisine of your choice: \n")
        dietary_restriction = input("Enter the diet you follow or restricted to: \n")                          
        recommended_items = allergy_based_recommender(cuisine, dietary_restriction)
        counter = 1
        for item, instructions in recommended_items.items():
            print(str(counter) + '. ' + item + ': ' + instructions + '\n')
            counter += 1
        
        
    elif option == "8":
        cuisine = input("Enter the cuisine of your choice: \n")
        utensil = input("Enter the utensil are restricted to use: \n")  #utensil may be dish,pan,bowl,cooker,glass etc                        
        recommended_items = utensil_restriction_based_recommender(cuisine, utensil)
        counter = 1
        for item, instructions in recommended_items.items():
            print(str(counter) + '. ' + item + ': ' + instructions + '\n')
            counter += 1
        
        
        
    elif option == "9":
        cuisine = input("Enter the cuisine of your choice: \n")
        prep_time = int(input("Enter the time of preparation you prefer: \n"))
        recommended_items = prep_time_restriction_based_recommender(cuisine,prep_time)
        counter = 1
        for item, instructions in recommended_items.items():
            print(str(counter) + '. ' + item + ': ' + instructions + '\n')
            counter += 1
            
    elif option == "10":
        cuisine = input("Enter the cuisine of your choice: \n")
        course = input("Enter the course of your choice: \n")
        recommended_items = course_restriction_based_recommender(cuisine,course)
        
        counter = 1
        for item, instructions in recommended_items.items():
            print(str(counter) + '. ' + item + ': ' + instructions + '\n')
            counter += 1

    else:
        print("Invalid choice! Please choose a number between 1-10.")

Welcome to the Recipe Recommender System!

Select the option from 1-10 based upon your preference:
1. Cuisine Based Recommender System
2. Course Based Recommender System
3. Diet Based Recommender System
4. Preparation Based Recommender System
5. Ingredients and Prep-Time Based Recommender System
6. Allergy Free and Cuisine Based Recommender System
7. Dietary-Restriction and Cuisine Based Recommender System
8. Utensil-Restriction and Cuisine Based Recommender System
9. Preparation Time and Cuisine Based Recommender System
10. Course and Cuisine Based Recommender System

Enter your choice (1-10): 6


Enter the cuisine of your choice: 
Indian
Enter the food you are allergic to: 
methi
1. Besan & Atta Ki Roti Recipe (Chickpea Flatbread): To begin making the Besan & Atta Ki Roti Recipe (Chickpea Flatbread), first get all the ingredients ready and keep aside.Combine all the ingredients into a large mixing bowl (except the oil or ghee). Add a little water at a time and knead to make a smooth 

## 1. Cuisine Based Recommender-System

In [71]:
def cuisine_based_recommender(cuisine, num_recommendations=5):
    selected_cols = ['name', 'cuisine','ingredients','instructions']
    df_subset = df[selected_cols]

    df_subset = df_subset[df_subset['cuisine'] == cuisine]

    tfidf = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf.fit_transform(df_subset['ingredients'])

    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

    indices = pd.Series(df_subset.index)
    indices_top_n = indices.nlargest(num_recommendations)

    recommended_items = df_subset.loc[indices_top_n, ['name', 'instructions']]
    recommended_items = recommended_items.set_index('name')

    recommended_items_dict = recommended_items.to_dict()['instructions']

    return recommended_items_dict



## 2. Course Based Recommender-System

In [72]:
def course_based_recommender(course, num_recommendations=5):
    selected_cols = ['name', 'course','ingredients','instructions']
    df_subset = df[selected_cols]

    df_subset = df_subset[df_subset['course'] == course]

    tfidf = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf.fit_transform(df_subset['ingredients'])

    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

    indices = pd.Series(df_subset.index)
    indices_top_n = indices.nlargest(num_recommendations)

    recommended_items = df_subset.loc[indices_top_n, ['name', 'instructions']]
    recommended_items = recommended_items.set_index('name')

    recommended_items_dict = recommended_items.to_dict()['instructions']

    return recommended_items_dict

## 3. Diet Based Recommender-System

In [73]:
def diet_based_recommender(diet, num_recommendations=5):
    # Select only the 'name' and 'diet' columns
    selected_cols = ['name', 'diet','ingredients','instructions']
    df_subset = df[selected_cols]

    # Filter by diet
    df_subset = df_subset[df_subset['diet'] == diet]

    # Generate TF-IDF vectors for the 'name' column
    tfidf = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf.fit_transform(df_subset['ingredients'])

    # Compute cosine similarity between all pairs of items
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

    # Get the indices of the top-n most similar items
    indices = pd.Series(df_subset.index)
    indices_top_n = indices.nlargest(num_recommendations)

    # Get the names and instructions of the recommended items
    recommended_items = df_subset.loc[indices_top_n, ['name', 'instructions']]
    recommended_items = recommended_items.set_index('name')

    # Convert the DataFrame to a dictionary of recommended items
    recommended_items_dict = recommended_items.to_dict()['instructions']

    return recommended_items_dict

## 4. Prep-Time Based Recommender-System

In [74]:
def prep_based_recommender(prep_time, num_recommendations=5):
    # Select only the 'name' and 'prep_time' columns
    selected_cols = ['name', 'prep_time','ingredients','instructions']
    df_subset = df[selected_cols]

    # Filter by prep_time
    df_subset = df_subset[df_subset['prep_time'] == prep_time]

    # Generate TF-IDF vectors for the 'name' column
    tfidf = TfidfVectorizer()
    tfidf_matrix = tfidf.fit_transform(df_subset['ingredients'])

    # Compute cosine similarity between all pairs of items
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

    # Get the indices of the top-n most similar items
    indices = pd.Series(df_subset.index)
    indices_top_n = indices.nlargest(num_recommendations)

    # Get the names and instructions of the recommended items
    recommended_items = df_subset.loc[indices_top_n, ['name', 'instructions']]
    recommended_items = recommended_items.set_index('name')

    # Convert the DataFrame to a dictionary of recommended items
    recommended_items_dict = recommended_items.to_dict()['instructions']

    return recommended_items_dict

## 5. Ingredients Based Recommendation-System

In [75]:
# Clean the 'ingredients' column and create a new column 'ingredients_clean'
df['ingredients_clean'] = df['ingredients'].str.replace('\d+g\s+', '').str.replace('\d+\s+\(.*?\)\s+', '').str.replace('\s+\(.+?\)\s+', '').str.replace('\s+\d+\s+', '').str.strip()

# Define the content-based recommendation function
def ingredients_based_recommender(ingredients, prep_time, num_recommendations=5):
    # Select only the 'name' and 'ingredients_clean' columns
    selected_cols = ['name', 'ingredients_clean','prep_time','instructions']
    df_subset = df[selected_cols]

    # Generate TF-IDF vectors for the 'ingredients_clean' column
    tfidf = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf.fit_transform(df_subset['ingredients_clean'])

    # Compute cosine similarity between all pairs of items
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

    # Get the indices of the top-n most similar items
    indices = pd.Series(df_subset.index)
    indices_top_n = indices.nlargest(num_recommendations)

    # Get the names and instructions of the recommended items
    recommended_items = df_subset.loc[indices_top_n, ['name', 'instructions']]
    recommended_items = recommended_items.set_index('name')

    # Convert the DataFrame to a dictionary of recommended items
    recommended_items_dict = recommended_items.to_dict()['instructions']

    return recommended_items_dict

## 6. Allergy Based Recommender-System

In [76]:
def allergy_based_recommender(cuisine, allergen, num_recommendations=5):
    # Select only the 'name', 'cuisine', 'ingredients' and 'instructions' columns
    selected_cols = ['name', 'cuisine', 'ingredients', 'instructions']
    df_subset = df[selected_cols]

    # Filter by cuisine
    df_subset = df_subset[df_subset['cuisine'] == cuisine]

    # Remove allergen from ingredients and instructions
    df_subset = df_subset[~df_subset['ingredients'].str.contains(allergen, case=False)]
    df_subset = df_subset[~df_subset['instructions'].str.contains(allergen, case=False)]

    # Generate TF-IDF vectors for the 'name' column
    tfidf = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf.fit_transform(df_subset['ingredients'])

    # Compute cosine similarity between all pairs of items
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

    # Get the indices of the top-n most similar items
    indices = pd.Series(df_subset.index)
    indices_top_n = indices.nlargest(num_recommendations)

    # Get the names and instructions of the recommended items
    recommended_items = df_subset.loc[indices_top_n, ['name', 'instructions']]

    recommended_items = recommended_items.set_index('name')

    # Convert the DataFrame to a dictionary of recommended items
    recommended_items_dict = recommended_items.to_dict()['instructions']

    return recommended_items_dict

## 7. Dietary-Restriction Based Recommender System

In [77]:
def dietary_restriction_based_recommender(cuisine, dietary_restriction, num_recommendations=5):
    # Select only the 'name' and 'cuisine' columns
    selected_cols = ['name', 'cuisine', 'diet', 'ingredients', 'instructions']
    df_subset = df[selected_cols]

    # Replace missing values in the 'diet' column with 'Not Specified'
    df_subset.loc[:, 'diet'] = df_subset['diet'].fillna('Not Specified')

    # Filter by cuisine and dietary restriction
    df_subset = df_subset[(df_subset['cuisine'] == cuisine) & (df_subset['diet'] == dietary_restriction)]

    # Generate TF-IDF vectors for the 'name' column
    tfidf = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf.fit_transform(df_subset['ingredients'])

    # Compute cosine similarity between all pairs of items
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

    # Get the indices of the top-n most similar items
    indices = pd.Series(df_subset.index)
    indices_top_n = indices.nlargest(num_recommendations)

    # Get the names and instructions of the recommended items
    recommended_items = df_subset.loc[indices_top_n, ['name', 'instructions']]
    recommended_items = recommended_items.set_index('name')

    # Convert the DataFrame to a dictionary of recommended items
    recommended_items_dict = recommended_items.to_dict()['instructions']

    return recommended_items_dict

## 8. Utensil-Restriction Based Recommender-System

In [78]:
def utensil_restriction_based_recommender(cuisine, utensil, num_recommendations=5):
    # Select only the 'name' and 'cuisine' columns
    selected_cols = ['name', 'cuisine','ingredients','instructions']
    df_subset = df[selected_cols]

    # Filter by cuisine
    df_subset = df_subset[df_subset['cuisine'] == cuisine]

    # Filter out items containing the specified utensil in ingredients or instructions
    df_subset = df_subset[~(df_subset['ingredients'].str.contains(utensil, case=False)) & ~(df_subset['instructions'].str.contains(utensil, case=False))]

    # Generate TF-IDF vectors for the 'name' column
    tfidf = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf.fit_transform(df_subset['ingredients'])

    # Compute cosine similarity between all pairs of items
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

    # Get the indices of the top-n most similar items
    indices = pd.Series(df_subset.index)
    indices_top_n = indices.nlargest(num_recommendations)

    # Get the names and instructions of the recommended items
    recommended_items = df_subset.loc[indices_top_n, ['name', 'instructions']]
    recommended_items = recommended_items.set_index('name')

    # Convert the DataFrame to a dictionary of recommended items
    recommended_items_dict = recommended_items.to_dict()['instructions']

    return recommended_items_dict

## 9. PrepTime-Restriction Based Recommender-System

In [79]:
def prep_time_restriction_based_recommender(cuisine, prep_time, num_recommendations=5):
    # Select only the 'name', 'cuisine', 'ingredients', 'instructions', and 'prep_time' columns
    selected_cols = ['name', 'cuisine', 'ingredients', 'instructions', 'prep_time']
    df_subset = df[selected_cols]

    # Filter by cuisine and prep_time
    df_subset = df_subset[(df_subset['cuisine'] == cuisine) & (df_subset['prep_time'] <= prep_time)]

    # Generate TF-IDF vectors for the 'name' column
    tfidf = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf.fit_transform(df_subset['ingredients'])

    # Compute cosine similarity between all pairs of items
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

    # Get the indices of the top-n most similar items
    indices = pd.Series(df_subset.index)
    indices_top_n = indices.nlargest(num_recommendations)

    # Get the names and instructions of the recommended items
    recommended_items = df_subset.loc[indices_top_n, ['name', 'instructions']]
    recommended_items = recommended_items.set_index('name')

    # Convert the DataFrame to a dictionary of recommended items
    recommended_items_dict = recommended_items.to_dict()['instructions']

    return recommended_items_dict

## 10. Course-Restriction Based Recommender-System

In [80]:
def course_restriction_based_recommender(cuisine, course, num_recommendations=5):
    # Select only the 'name', 'cuisine', 'course', 'ingredients', and 'instructions' columns
    selected_cols = ['name', 'cuisine', 'course', 'ingredients', 'instructions']
    df_subset = df[selected_cols]

    # Filter by cuisine and course
    df_subset = df_subset[(df_subset['cuisine'] == cuisine) & (df_subset['course'] == course)]

    # Generate TF-IDF vectors for the 'name' column
    tfidf = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf.fit_transform(df_subset['ingredients'])

    # Compute cosine similarity between all pairs of items
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

    # Get the indices of the top-n most similar items
    indices = pd.Series(df_subset.index)
    indices_top_n = indices.nlargest(num_recommendations)

    # Get the names and instructions of the recommended items
    recommended_items = df_subset.loc[indices_top_n, ['name', 'instructions']]
    recommended_items = recommended_items.set_index('name')

    # Convert the DataFrame to a dictionary of recommended items
    recommended_items_dict = recommended_items.to_dict()['instructions']

    return recommended_items_dict

## ∞. Evaluation

### 1. Cuisine-based

In [ ]:
test_set = df
selected_cuisines = ['Indian', 'North Indian Recipes', 'Mangalorean', 'Kerala Recipes', 'Konkan', 'South Indian Recipes', 'Tamil Nadu', 'Bengali Recipes', 'Andhra', 'Coorg', 'Hyderabadi', 'Kashmiri', 'Punjabi', 'Gujarati Recipes\ufeff', 'Coastal Karnataka', 'Karnataka', 'Goan Recipes', 'Sichuan', 'Udupi', 'Chettinad', 'Maharashtrian Recipes', 'Mughlai', 'Malvani', 'North East India Recipes', 'Rajasthani', 'Parsi Recipes', 'Bihari', 'Awadhi', 'North Karnataka', 'Assamese', 'Pakistani', 'Himachal', 'Middle Eastern', 'Oriya Recipes', 'Uttarakhand-North Kumaon ', 'Lucknowi', 'Sindhi', 'Indo Chinese', 'Uttar Pradesh', 'Kongunadu', 'Chinese', 'Haryana', 'South Karnataka', 'Fusion', 'Malabar', 'Nepalese', 'Continental', 'Jharkhand', 'Sri Lankan', 'Asian', 'Thai', 'Arab', 'Nagaland', 'Afghan', 'African', 'Korean']
test_set = test_set[test_set['cuisine'].isin(selected_cuisines)]

def mapk(actual, predicted, k=5):
    if len(predicted) > k:
        predicted = predicted[:k]
    score = 0.0
    num_hits = 0.0
    for i, p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)
    if not actual:
        return 0.0
    return score / min(len(actual), k)


def ndcg_k(actual, predicted, k=5):
    idcg = 0.0
    for i in range(min(k, len(actual))):
        idcg += 1.0 / np.log2(i+2)
    dcg = 0.0
    for i, p in enumerate(predicted[:k]):
        if p in actual:
            dcg += 1.0 / np.log2(i+2)
    if not actual:
        return 0.0
    return dcg / idcg


def evaluate_recommendations(actual, recommended, k=5):
    predicted = list(recommended.values())[:k]
    if not actual:
        mapk_score = 0.0
    else:
        mapk_score = mapk(actual, predicted, k)
    dcg = 0.0
    idcg = 0.0
    for i, item in enumerate(recommended.items()):
        if i >= k:
            break
        if item[1] in actual:
            dcg += 1.0 / np.log2(i+2)
        if i == 0:
            idcg += 1.0
        else:
            idcg += 1.0 / np.log2(i+2)
    ndcg_k_score = dcg / idcg
    return mapk_score, ndcg_k_score


y_true = test_set['cuisine'].values
test_set_subset = test_set.head(149)
y_true = test_set_subset['cuisine'].values
y_pred = []
for i, row in test_set_subset.iterrows():
    cuisine = row['cuisine']
    recommended_items = cuisineBased(cuisine)
    if recommended_items:
        predicted_cuisine = max(set([df.loc[df['name'] == name, 'cuisine'].values[0] for name in recommended_items.keys()]), key = lambda x: list(recommended_items.values()).count(x))
    else:
        predicted_cuisine = cuisine
    y_pred.append(predicted_cuisine)
accuracy = accuracy_score(y_true, y_pred)
print('Accuracy:', accuracy)
cm = confusion_matrix(y_true, y_pred)
total_predictions = sum(sum(cm))
correct_predictions = sum(cm.diagonal())
print("Total Predictions:", total_predictions)
print("Correct Predictions:", correct_predictions)
print("Confusion Matrix:\n", cm)

Accuracy: 1.0
Total Predictions: 149
Correct Predictions: 149
Confusion Matrix:
 [[ 7  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0]
 [ 0  6  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0]
 [ 0  0  6  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0]
 [ 0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0]
 [ 0  0  0  0  3  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0]
 [ 0  0  0  0  0  6  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0]
 [ 0  0  0  0  0  0  4  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0]
 [ 0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0]
 [ 0  0  0  0  0  0  0  0 25  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0]
 [ 0  0  0  0  0  0  0  0  0  5  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0]
 [ 0  0  0  0  0  0  0  0  0  0  2  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0]
 [ 0  

### 2. Course-Based

In [ ]:
def course_based_recommender(course, num_recommendations=5):
    selected_cols = ['name', 'course','ingredients','instructions']
    df_subset = df[selected_cols]
    df_subset = df_subset[df_subset['course'] == course]
    tfidf = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf.fit_transform(df_subset['ingredients'])
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
    indices = pd.Series(df_subset.index)
    indices_top_n = indices.nlargest(num_recommendations)
    recommended_items = df_subset.loc[indices_top_n, ['name', 'instructions']]
    recommended_items = recommended_items.set_index('name')
    recommended_items_dict = recommended_items.to_dict()['instructions']

    actual_labels = []
    for item in recommended_items.index:
        actual_labels.append(df.loc[df['name'] == item, 'course'].values[0])
    predicted_labels = [course] * num_recommendations
    accuracy = accuracy_score(actual_labels, predicted_labels)
    cm = confusion_matrix(actual_labels, predicted_labels)

    return accuracy, cm


course = 'Side Dish'
accuracy, cm = course_based_recommender(course)
print("Accuracy:", accuracy)
total_predictions = sum(sum(cm))
correct_predictions = sum(cm.diagonal())
print("Total Predictions:", total_predictions)
print("Correct Predictions:", correct_predictions)
print("Confusion matrix: \n", cm)

Accuracy: 1.0
Total Predictions: 5
Correct Predictions: 5
Confusion matrix: 
 [[5]]


### 3. Diet-Based

In [ ]:
def diet_based_recommender(diet, num_recommendations=5):
    selected_cols = ['name', 'diet','ingredients','instructions']
    df_subset = df[selected_cols]
    df_subset = df_subset[df_subset['diet'] == diet]
    tfidf = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf.fit_transform(df_subset['ingredients'])
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
    indices = pd.Series(df_subset.index)
    indices_top_n = indices.nlargest(num_recommendations)
    recommended_items = df_subset.loc[indices_top_n, ['name', 'instructions']]
    recommended_items = recommended_items.set_index('name')
    recommended_items_dict = recommended_items.to_dict()['instructions']
    actual_labels = []
    for item in recommended_items.index:
        actual_labels.append(df.loc[df['name'] == item, 'diet'].values[0])
    predicted_labels = [diet] * num_recommendations
    accuracy = accuracy_score(actual_labels, predicted_labels)
    cm = confusion_matrix(actual_labels, predicted_labels)
    num_correct_predictions = sum([1 for i in range(num_recommendations) if actual_labels[i] == diet])
    return accuracy, cm, num_correct_predictions, num_recommendations


diet = 'Vegetarian'
accuracy, cm, num_correct_predictions, num_recommendations = diet_based_recommender(diet)
print("Accuracy:", accuracy)
print("Total Predictions:", num_recommendations)
print("Correct Predictions:", num_correct_predictions)
print("Confusion matrix: \n", cm)

Accuracy: 1.0
Total Predictions: 5
Correct Predictions: 5
Confusion matrix: 
 [[5]]
